# Projeto Zyfra – Modelo de Previsão para Extração de Ouro

É hora de lidar com um problema real de ciência de dados do campo da mineração de ouro. Este projeto foi fornecido pela Zyfra (os materiais estão em inglês).

## Descrição da Tarefa

Prepare um protótipo de um modelo de aprendizado de máquina para o Zyfra. A empresa desenvolve soluções de eficiência para a indústria pesada.

O modelo deve prever a quantidade de ouro puro extraído do minério de ouro. Você tem os dados sobre a extração e a purificação.

## Descrição do Projeto

Os dados são indexados com a data e hora da aquisição (característica data). Os parâmetros que estão próximos uns dos outros em termos de tempo geralmente são semelhantes.

Alguns parâmetros não estão disponíveis porque foram medidos e/ou calculados muito mais tarde. Por isso, algumas das características presentes no conjunto de treinamento podem estar ausentes do conjunto de teste. O conjunto de teste também não contém objetivos.

O conjunto de dados de origem contém os conjuntos de treinamento e teste com todas as características.

Você tem os dados brutos, recebidos diretamente do cliente. Antes de construir o modelo, verifique a exatidão dos dados. Para isso, use nossas instruções.

---

## 1. PREPARAÇÃO DOS DADOS

# 1.1 Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np

# Configurando o Pandas a mostrar o máximo de colunas e linhas
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", None)

# Impedir avisos
import warnings
warnings.filterwarnings("ignore")

# Criação gráfica
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt

# Modelo
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

# Metricas
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, make_scorer

# Pipeline
from sklearn.pipeline import Pipeline

# Column Transformer
from sklearn.compose import ColumnTransformer

# Pre processamento
from sklearn.preprocessing import StandardScaler

# Model Selection
from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
# Carregue os arquivos de dados em diferentes DataFrames

# Local Path
local_gold_recov_full_path = "gold_recovery_full.csv"
local_gold_recov_test_path = "gold_recovery_test.csv"
local_gold_recov_train_path = "gold_recovery_train.csv"

# Cloud Path
gold_recov_full_path = "/datasets/gold_recovery_full.csv"
gold_recov_test_path = "/datasets/gold_recovery_test.csv"
gold_recov_train_path = "/datasets/gold_recovery_train.csv"

try:
    df_full = pd.read_csv(local_gold_recov_full_path, sep=",", parse_dates=True)
    df_test = pd.read_csv(local_gold_recov_test_path, sep=",", parse_dates=True)
    df_train = pd.read_csv(local_gold_recov_train_path, sep=",", parse_dates=True)

except FileNotFoundError:
    print(
        f"O arquivo CSV não foi encontrado em {local_gold_recov_full_path}. Tentando o caminho {gold_recov_full_path}..."
    )

    print(
        f"O arquivo CSV não foi encontrado em {local_gold_recov_test_path}. Tentando o caminho {gold_recov_test_path}..."
    )

    print(
        f"O arquivo CSV não foi encontrado em {local_gold_recov_train_path}. Tentando o caminho {gold_recov_train_path}..."
    )

    try:
        df_full = pd.read_csv(gold_recov_full_path, sep=",", parse_dates=True)
        df_test = pd.read_csv(gold_recov_test_path, sep=",", parse_dates=True)
        df_train = pd.read_csv(gold_recov_train_path, sep=",", parse_dates=True)

    except FileNotFoundError:
        print(
            f"O arquivo CSV não foi encontrado em {gold_recov_full_path}. Nenhum arquivo encontrado."
        )

        print(
            f"O arquivo CSV não foi encontrado em {gold_recov_test_path}. Nenhum arquivo encontrado."
        )

        print(
            f"O arquivo CSV não foi encontrado em {gold_recov_train_path}. Nenhum arquivo encontrado."
        )

In [ ]:
# Checando o shape dos dataframes

df_full.shape, df_train.shape, df_test.shape

Podemos perceber de cara, que há uma diferença de colunas usadas entre o dataset de treino (87) e o dataset de teste (53).

In [ ]:
# Checando se DataFrame foi importado
df_full.head()

In [ ]:
# Checando se DataFrame foi importado
df_test.head()

In [ ]:
# Checando se DataFrame foi importado
df_train.head()

## 1.2 Checando se a quantidade retirada foi calculada corretamente.

In [ ]:
# Copiando o dataframe de treinamento
new_df = df_train.copy()

# Separando as colunas necessárias para análise
subset_columns = subset = [
    "rougher.output.recovery",
    "rougher.output.concentrate_au",
    "rougher.input.feed_au",
    "rougher.output.tail_au",
]

# Crie um novo DataFrame com o subset das colunas necessárias
df_train_recovery_calc_subset = new_df[subset_columns]
df_train_recovery_calc_subset.head()

In [ ]:
df_train_recovery_calc_subset.info()

Temos dados faltantes, como podemos ver a abaixo.

In [ ]:
df_train_recovery_calc_subset.isna().sum()

Vamos fazer o drop desses dados faltantes.

In [ ]:
# Dropna
df_train_recovery_calc_subset = df_train_recovery_calc_subset.dropna()

# Checando se os dados foram apagados
df_train_recovery_calc_subset.isna().sum()

Com as linhas de dados faltantes apagados, podemos calcular o valor de recovery, de acordo com a fórmula abaixo:

In [ ]:
# Função para calcular a recuperação de ouro
def calculate_recovery(df, C, F, T):
    """
    Função para calcular a recuperação de ouro.

    Para calcular rougher recovery, a função precisa dos seguintes imputs:
    - df = DataFrame onde a coluna está presente
    - C = proporção de ouro no concentrado logo após a flotação
    - F = a proporção de ouro alimentado no sistema antes da flotação
    - T = a proporção de ouro nos restos de minério bruto logo após a flotação

    Esta função retorna o percentual de recuperação do concentrado de ouro após a etapa de purificação.
    """

    recovery = (df[C] * (df[F] - df[T])) / (df[F] * (df[C] - df[T])) * 100

    return recovery

In [ ]:
# calculando o valor de recovery
calculated_recovery = calculate_recovery(
    df_train_recovery_calc_subset,
    "rougher.output.concentrate_au",
    "rougher.input.feed_au",
    "rougher.output.tail_au",
)
# Mostrando o resultado das 10 primeiras amostras
calculated_recovery.head(10)

Agora podemos calcular o Mean Absolute Error (MAE) entre o cálculo gravado em calculated_recovery e os valores da feature df_train_recovery_calc_subset['rougher.output.recovery']

In [ ]:
# Calculo Mean Absolute Error
mae = mean_absolute_error(
    df_train_recovery_calc_subset["rougher.output.recovery"], calculated_recovery
)

print(f"A diferença entre os cálculos e o valores originais são de: {mae}")

O fato de o erro médio absoluto (MAE) entre nossos valores de recuperação calculados e os valores de recurso fornecidos, ser de 9.303415616264301e-15, ou seja, quase zero, sugere que nossos cálculos são quase idêntico aos valores de recurso fornecidos para rougher.output.recovery.

### 1.3 Analisando recursos que não estão disponíveis no conjunto de testes

In [ ]:
# Olhando as colunas que não foram usadas no dataframe de teste
missed_columns = list(set(df_train.columns) - set(df_test.columns))

In [ ]:
missed_columns

Aparentemente, todos os recursos que não estão incluídos no conjunto de teste são do tipo "_output_" e "calculation"

In [ ]:
# Tamanho das colunas faltantes
len(missed_columns)

São 34 colunas faltantes nop conjunto de teste.

In [ ]:
df_full[missed_columns].dtypes

Podemos observar que todas as colunas faltantes são do tipo float

### 1.4 Preparação dos dados

#### 1.4.1 Checando dados faltosos

In [ ]:
def get_missing_values_report(dataset_name: str, df: pd.DataFrame):
    """
    Esta função:
     1. Primeiro, usa um DataFrame como entrada.
     2. calcula as informações de valor ausentes e cria um relatório.
     3. imprime as informações e o relatório.
    """
    # Calcula o número de valores faltantes em todas as colunas do DataFrame
    number_missing_values = df.isna().sum()

    # Obtém o número total de registros/observações no DataFrame
    number_of_rows = df.shape[0]

    # Calcula a porcentagem de valores faltantes em todas as colunas do DataFrame
    percentage_of_missing_values = round(
        (number_missing_values / number_of_rows) * 100, 2
    )

    # Crie um novo DataFrame com número de valores ausentes e porcentagem de valores ausentes em todas as colunas
    df_missing_values = pd.concat(
        [number_missing_values, percentage_of_missing_values], axis=1
    )
    df_missing_values = df_missing_values.rename(
        columns={0: "No of Missing Values", 1: "Percentage of Missing Values"}
    )

    # Classifica o novo DataFrame de acordo com a porcentagem de valores ausentes em ordem decrescente
    df_missing_values = df_missing_values.sort_values(
        by="Percentage of Missing Values", ascending=False
    )

    # Imprime as estatísticas de valor faltantes
    print("----------------------------------------------------------")
    print(f"        {dataset_name} Missing Values Report")
    print("----------------------------------------------------------")
    print(f"Total number of rows: {number_of_rows}")
    print(f"Total number of columns: {df.shape[1]}")
    print()
    print("Informações sobre o número e porcentagem de valores ausentes no DataFrame")
    display(df_missing_values)

In [ ]:
# Create a list of the DataFrames
list_of_df = [
    {"name": "Training Dataset", "data": df_train},
    {"name": "Test Dataset", "data": df_test},
    {"name": "Source Dataset", "data": df_full},
]
for item in list_of_df:
    get_missing_values_report(item["name"], item["data"])

Podemos ver que em todos os datasets, temos dados faltantes. Apenas as colunas ["primary_cleaner.input.feed_size", "date"] que possuem todos os dados. 

- Como na descrição do projeto fala: 
  
      Os dados são indexados com a data e hora da aquisição (característica data). Os parâmetros que estão próximos uns dos outros em termos de tempo geralmente são semelhantes.

Logo, podemos preencher os dados ausentes com os valores próximos, porém, não pode ser feito para as variáveis objetivos.

Vamos corrigir as colunas que possuem dados faltantes!

In [ ]:
# Função para preencher os valores ausentes no DataFrame

target_cols = ["rougher.output.recovery", "final.output.recovery"]


def forward_fill_missing_values(df: pd.DataFrame):
    for col in df:
        if col not in target_cols:
            df[col].fillna(method="ffill", axis=0, inplace=True)

In [ ]:
# Preencher os valores faltantes
forward_fill_missing_values(df_train)
forward_fill_missing_values(df_test)
forward_fill_missing_values(df_full)

Como não foi preenchido os valores ausentes para as colunas objetivos, vamos eliminar as linhas dessas colunas que têm valores ausentes, para poder estimar os modelos de ML mais pra frente.

In [ ]:
# Drop dos valores ausentes no df_train
df_train = df_train.dropna(subset=["final.output.recovery", "rougher.output.recovery"])

# Checando se ainda consta algum valor ausente no df_train
df_train[["final.output.recovery", "rougher.output.recovery"]].isna().sum()

In [ ]:
# Drop dos valores ausentes no df_full
df_full = df_full.dropna(subset=["final.output.recovery", "rougher.output.recovery"])

# Checando se ainda consta algum valor ausente no df_full
df_full[["final.output.recovery", "rougher.output.recovery"]].isna().sum()

In [ ]:
# Checando de novo os valores nulos
for item in list_of_df:
    item["data"].info()

#### 1.4.2 Checando duplicados

In [ ]:
# Checando os dados duplicados no df_train
df_train.duplicated().sum()

In [ ]:
# Checando os dados duplicados no df_test
df_test.duplicated().sum()

In [ ]:
# Checando os dados duplicados no df_full
df_full.duplicated().sum()

Não existem dados duplicados em nenhum dos dataframes!

#### 1.4.3 Checandos os datatypes

Como pode ser visto anteriormente, a coluna "date" não está como datetime. Vamos alterar!

In [ ]:
# Convertendo a coluna datepara datetime
df_train["date"] = pd.to_datetime(df_train["date"])

# Print info
df_train.info()

In [ ]:
# Convertendo a coluna datepara datetime
df_test["date"] = pd.to_datetime(df_test["date"])

# Print info
df_test.info()

In [ ]:
# Convertendo a coluna datepara datetime
df_full["date"] = pd.to_datetime(df_full["date"])

# Print info
df_full.info()

Agora temos os dados limpos, os dados ausentes preenchidos e com os tipos corretos. Podemos prosseguir com as análises dos dados!

## 2. ANÁLISE DE DADOS

### 2.1 Analisando como a concentração de metais (Au, Ag, Pb) muda dependendo do estágio de purificação

Podemos ver que temos possíveis 4 valores de estágios: rougher, primary_cleaner, secondary_cleaner e final. Porém dos metais concentrados, no dataframe df_train, não apresenta a variável secondary_cleaner.

Dessa forma, analisaremos os 3 estágios: rougher, primary_cleaner e final, juntamente com os dados de input dos metais brutos.

In [ ]:
# Definindo os estágios
stages = [
    "rougher.input.feed",
    "rougher.output.concentrate",
    "primary_cleaner.output.concentrate",
    "final.output.concentrate",
]

# Criação de dicionário para cada metal
metals = {"_au": "Ouro", "_ag": "Prata", "_pb": "Chumbo"}


# Criação subplots para visualizar a concentração de cada metal
def plot_metal_concentration(df: pd.DataFrame, df_name: str):
    """
    Essa função:

    1. Faz o subplot da quantidade de gráficos que precisaremos, i.e., quantidade de metais da lista.

    2. plota o grafico de kernek Density Estimate (KDE) para visualizar a distribuição das observações no dataset, ao longo do histograma, de cada um dos metrais da lista e de cada estágio de purificação.
    """

    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 14), sharey=False)
    fig.suptitle(
        f"Mudança na Concentração de Metal por Estágio de Purificação (Densidade) \n\nDataSet: {df_name}",
        fontsize=15,
        y=1,
    )

    for metal in metals.keys():
        for stage in stages:
            ax = list(metals.keys()).index(metal)
            sns.kdeplot(
                df[(stage + metal)], ax=axes[ax], shade="fill", label=(stage + metal)
            )
            axes[ax].legend()
            axes[ax].set_xlabel(
                "Concentração de " + metals[metal] + " (%)", fontsize=12, labelpad=10
            )
            axes[ax].xaxis.set_label_position("top")
            axes[ax].set_ylabel("Densidade", fontsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
# Plotar a distribuição das concentrações de metais (Au, Ag, Pb) para cada estágio de purificação para df_full
plot_metal_concentration(df_full, "df_full")

In [ ]:
# Plotar a distribuição das concentrações de metais (Au, Ag, Pb) para cada estágio de purificação para df_train
plot_metal_concentration(df_train, "df_train")

Tanto no dataset df_full quanto no df_train, as observações são bem próximas, ou seja, a mudança na concentração de metal por estágio de purificação, são bem parecidas. Porém, olhando individualmente cada metal, há diferenças como:

- **Ouro**: A cada etapa do processo de purificação, a concentração de ouro aumenta.

- **Prata**: Após o processo de "feed", a concentração de prata aumenta, mas diminui a cada processo subsequente. O resultado final, "final.output.concentrate" é de uma concentração de prata inferior à concentração da mistura inicial do minério.

- **Chumbo**: Parece que a concentração de chumbo aumenta até a o segundo estágio do processo de limpeza, "rougher.output.concentrate", onde permanece aproximadamente igual à concentração final do processo de limpeza.

Podemos observar também, que há outliers nos dados. Pois, para cada etapa do processo, há casos em que a concentração de cada metal é 0.

#### 2.2 Comparando a distribuição do tamanho das partículas do minério no conjunto de treinamento e no conjunto de teste

In [ ]:
fig = plt.figure(figsize=(15, 6))
fig.suptitle(
    "Distribuição dos valores Primary Cleaner e Rougher Input Feed_Size (Densidade)",
    fontsize=15,
    y=1,
)

sns.kdeplot(
    df_train["primary_cleaner.input.feed_size"],
    fill=True,
    label="Train - Primary Feed Size",
)
sns.kdeplot(
    df_test["primary_cleaner.input.feed_size"],
    fill=True,
    label="Test - Primary Feed Size",
)
sns.kdeplot(
    df_train["rougher.input.feed_size"], fill=True, label="Train - Rougher Feed Size"
)
sns.kdeplot(
    df_test["rougher.input.feed_size"], fill=True, label="Test - Rougher Feed Size"
)
plt.legend(fontsize=12)
plt.xlabel("Valores Feed Size", fontsize=12, labelpad=10)
plt.ylabel("Densidade", fontsize=12)
plt.xlim(0, 150)
plt.tight_layout()
plt.show()

Como podemos ver no gráfico acima, as distribuições de valores para 'primary_cleaner.input.feed_size' e 'rougher.input.feed_size' são aproximadamente as mesmas para os conjuntos de dados de treinamento e teste

#### 2.3 Considerando as concentrações totais de todas as substâncias em diferentes estágios. Há anomalias em ambas amostras?

In [ ]:
def calc_and_plot_total_concentrations(
    df: pd.DataFrame, stage: str, list_of_columns: list
):
    """
    Essa função faz:
    1. Calcula o total da concentração de todas as substâncias
    2. Plota o histograma
    3. Plota o boxplot para analisar os outliers
    """

    # Calculo do total de concentração das substâncias
    df[stage] = df[list_of_columns].sum(axis=1)

    # Set the plot styles
    sns.set(rc={"figure.figsize": (16.0, 8.0)})
    sns.set(font_scale=1.2)

    # Plot histograma
    sns.histplot(df[stage], bins=100, stat="frequency", kde=True)
    plt.title(
        f"Distribuição Total da Concentração de Todos os Metais no Estágio: {stage}"
    )
    plt.xlabel(stage)
    plt.show()

    # set the boxplot styles
    flierprops = dict(
        marker="o",
        markersize=10,
        markeredgecolor="black",
        markerfacecolor="darkgreen",
        alpha=0.6,
    )
    meanprops = dict(marker="s", markerfacecolor="white", markeredgecolor="black")

    # Plot boxplot
    box_plot = sns.boxplot(
        data=df[stage],
        showmeans=True,
        orient="h",
        linewidth=2,
        flierprops=flierprops,
        meanprops=meanprops,
        palette="muted",
    )

    box_plot.set(
        xlabel=stage,
        title=f"Distribuição Total da Concentração de Todos os Metais no Estágio: {stage}",
    )

    return df

In [ ]:
# Criando lista de cada estágio de cada minério

feature_input = [
    "rougher.input.feed_au",
    "rougher.input.feed_ag",
    "rougher.input.feed_pb",
    "rougher.input.feed_sol",
]

feature_output = [
    "rougher.output.concentrate_au",
    "rougher.output.concentrate_ag",
    "rougher.output.concentrate_pb",
    "rougher.output.concentrate_sol",
]

feature_final = [
    "final.output.concentrate_au",
    "final.output.concentrate_ag",
    "final.output.concentrate_pb",
    "final.output.concentrate_sol",
]

##### 2.3.1 Full Dataset

- **ROUGHER INPUT FEED STAGE**

In [ ]:
df_full = calc_and_plot_total_concentrations(
    df_full, "feed_total_concentractions", feature_input
)

- **ROUGHER OUTPUT STAGE**

In [ ]:
df_full = calc_and_plot_total_concentrations(
    df_full, "rougher_output_total_concentractions", feature_output
)

- **FINAL OUTPUT STAGE**

In [ ]:
df_full = calc_and_plot_total_concentrations(
    df_full, "final_total_concentractions", feature_final
)

#### Conclusão

**ROUGHER INPUT FEED STAGE**

- As concentrações totais de todas as substâncias no conjunto de origem são inclinadas à esquerda, pois o pico da distribuição está no lado direito e a média é menor que a mediana.

- Existem muitos valores discrepantes, mas há um grande aumento incomum na concentração de todas as substâncias em 0.

- Removeremos todas as observações no conjunto de origem - df_full onde df_full['feed_total_concentractions'] < 0,8


**ROUGHER OUTPUT STAGE**

- As concentrações totais de todas as substâncias no conjunto de origem são inclinadas à esquerda, pois o pico da distribuição está no lado direito e a média é menor que a mediana.

- Existem muitos valores discrepantes, mas há um grande aumento incomum na concentração de todas as substâncias em 0.

- Removeremos todas as observações no conjunto de origem - df_full onde df_full['rougher_output_total_concentractions'] < 0,8


**FINAL OUTPUT STAGE**

- As concentrações totais de todas as substâncias no conjunto de origem são inclinadas à esquerda, pois o pico da distribuição está no lado direito e a média é menor que a mediana.

- Existem muitos valores discrepantes, mas há um grande aumento incomum na concentração de todas as substâncias em 0.

- Removeremos todas as observações no conjunto de origem - df_full onde df_full['final_total_concentractions'] < 0,8

##### 2.3.2 Train Dataset

- **ROUGHER INPUT FEED STAGE**

In [ ]:
df_train = calc_and_plot_total_concentrations(
    df_train, "feed_total_concentractions", feature_input
)

- **ROUGHER OUTPUT STAGE**

In [ ]:
df_train = calc_and_plot_total_concentrations(
    df_train, "rougher_output_total_concentractions", feature_output
)

- **FINAL OUTPUT STAGE**

In [ ]:
df_train = calc_and_plot_total_concentrations(
    df_train, "final_total_concentractions", feature_final
)

#### Conclusão

**ROUGHER INPUT FEED STAGE**

- As concentrações totais de todas as substâncias no conjunto de origem são inclinadas à esquerda, pois o pico da distribuição está no lado direito e a média é menor que a mediana.

- Existem muitos valores discrepantes, mas há um grande aumento incomum na concentração de todas as substâncias em 0.

- Removeremos todas as observações no conjunto de origem - df_train onde df_train['feed_total_concentractions'] < 0,8


**ROUGHER OUTPUT STAGE**

- As concentrações totais de todas as substâncias no conjunto de origem são inclinadas à esquerda, pois o pico da distribuição está no lado direito e a média é menor que a mediana.

- Existem muitos valores discrepantes, mas há um grande aumento incomum na concentração de todas as substâncias em 0.

- Removeremos todas as observações no conjunto de origem - df_train onde df_train['rougher_output_total_concentractions'] < 0,8


**FINAL OUTPUT STAGE**

- As concentrações totais de todas as substâncias no conjunto de origem são inclinadas à esquerda, pois o pico da distribuição está no lado direito e a média é menor que a mediana.

- Existem muitos valores discrepantes, mas há um grande aumento incomum na concentração de todas as substâncias em 0.

- Removeremos todas as observações no conjunto de origem - df_train onde df_train['final_total_concentractions'] < 0,8

##### 2.3.3 Test Dataset

Como o conjunto de dados de teste - df_test não possui todos os recursos de saída, temos apenas as colunas abaixo que vêm do estágio Raw Feed:

- rougher.input.feed_ag
- rougher.input.feed_pb
- rougher.input.feed_sol
- rougher.input.feed_au

Podemos analisar as concentrações totais de todas as substâncias no conjunto de testes - df_test apenas para o estágio Raw Feed e encontrar e remover as anomalias.

- **ROUGHER INPUT FEED STAGE**

In [ ]:
df_test = calc_and_plot_total_concentrations(
    df_test, "feed_total_concentractions", feature_input
)

#### Conclusão

**ROUGHER INPUT FEED STAGE**

- As concentrações totais de todas as substâncias no conjunto de origem são inclinadas à esquerda, pois o pico da distribuição está no lado direito e a média é menor que a mediana.

- Existem muitos valores discrepantes, mas há um grande aumento incomum na concentração de todas as substâncias em 0.

- Removeremos todas as observações no conjunto de origem - df_test onde df_test['feed_total_concentractions'] < 0,8


##### 2.3.4 Removendo Outliers

In [ ]:
# Filtrando os datasets com as conclusões encontradas anteriormente
df_full = df_full[
    (df_full["feed_total_concentractions"] >= 0.8)
    & (df_full["rougher_output_total_concentractions"] >= 0.8)
    & (df_full["final_total_concentractions"] >= 0.8)
]

df_train = df_train[
    (df_train["feed_total_concentractions"] >= 0.8)
    & (df_train["rougher_output_total_concentractions"] >= 0.8)
    & (df_train["final_total_concentractions"] >= 0.8)
]

df_test = df_test[df_test["feed_total_concentractions"] >= 0.8]

# Obtendo o shape dos datasets filtrados
df_full.shape, df_train.shape, df_test.shape

#### 2.4 Preparando os Dados Finais para os Modelos

In [ ]:
# Fazendo cópias dos dataframes
new_df_full = df_full.copy()
new_df_train = df_train.copy()
new_df_test = df_test.copy()

In [ ]:
# Removendo as colunas criadas na seção anterior no dataframe new_df_train
columns_to_drop = [
    "feed_total_concentractions",
    "rougher_output_total_concentractions",
    "final_total_concentractions",
]
new_df_train = new_df_train.drop(columns_to_drop, axis=1)

# Removendo a coluna criada na seção anterior no dataframe new_df_test
new_df_test = new_df_test.drop(["feed_total_concentractions"], axis=1)

new_df_train.info(), new_df_test.info()

Agora vamos remover as colunas do dataset de treinamento que não estão presente no dataset de teste. Porém não podemos remover as 2 colunas objetivos, que são: "rougher.output.recovery" e "final.output.recovery"

In [ ]:
# Mostrando novamente as colunas faltantes no dataset df_test
missed_columns

In [ ]:
# Removendo rougher.output.recovery e final.output.recovery da lista missed_columns
missed_columns.remove("rougher.output.recovery")
missed_columns.remove("final.output.recovery")

In [ ]:
# Drop colunas que não estão presentes no dataset df_test
new_df_train = new_df_train.drop(missed_columns, axis=1)

In [ ]:
# Listando as colunas do new_df_train
display(new_df_train.columns)

In [ ]:
# new_df_train shape
new_df_train.shape

Agora, precisamos obter os valores das variáveis objetivo no conjunto de dados de origem (df_full)

Temos dois alvos: rougher.output.recovery e final.output.recovery. 

Mas, no conjunto de testes (df_test) não possui nenhum deles, pois são colunas relacionadas à saída. Precisaremos de ambos os recursos para fins de validação.

Sabemos que temos dados completos no conjunto de origem (df_full) e temos uma coluna em comum entre o conjunto de dados df_full e df_test, que é a coluna "date". Podemos usá-lo para mesclar os conjuntos de dados e recuperar os valores dos alvos.

In [ ]:
# Merge new_df_full e new_df_test pela coluna "date"
df_source_to_merge = new_df_full[
    ["date", "rougher.output.recovery", "final.output.recovery"]
]
df_test_derived = pd.merge(df_source_to_merge, new_df_test, how="inner", on="date")

In [ ]:
# df_test_derived shape
df_test_derived.shape

In [ ]:
# df_test_derived info
df_test_derived.info()

Ultima coisa a se fazer, é fazer o drop da coluna "date" do new_df_train e new_df_test, pois não são necessários para o modelo.

In [ ]:
# Drop date de  df_test_derived e new_df_train
df_test_derived = df_test_derived.drop(["date"], axis=1)
new_df_train = new_df_train.drop(["date"], axis=1)

df_test_derived.columns, new_df_train.columns

## 3. CONSTRUINDO O MODELO

Por conveniência e praticidade, vamos alterar os conjuntos de dados new_df_train e df_test_derived para df_train e df_test, respectivamente

In [ ]:
# renomeando os datasets
df_train = new_df_train
df_test = df_test_derived

In [ ]:
# Definir as features e target
targets = ["rougher.output.recovery", "final.output.recovery"]

features_train, target_train = df_train.drop(targets, axis=1), df_train[targets]
features_test, target_test = df_test.drop(targets, axis=1), df_test[targets]

In [ ]:
# features shape
features_train.shape, features_test.shape

In [ ]:
# targets shape
target_train.shape, target_test.shape

Vamos fazer o preprocessamento dos dados

In [ ]:
# Utilizando o StandardScale para padronizar os dados
numeric_columns = features_train.select_dtypes(include="number").columns.tolist()

# Criar o pipeline com StandardScaler
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

# Criar o pré-processador para as colunas numéricas
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_columns)]
)

# Aplicar o pré-processador aos dados de treino e teste
features_train_processed = preprocessor.fit_transform(features_train)
features_test_processed = preprocessor.transform(features_test)

#### 3.1 Calculo final sMAPE

- Cálculo da função sMAPE:

$$ sMAPE = {1 \over N} \sum_{i=1}^{N} {|y_i - \hat y_i |\over (|y_i| + |\hat y_i|) / 2} * 100\% $$

In [ ]:
# função para calcular smape
def calculate_smape(real: pd.Series, prediction: pd.Series):
    """
    Essa função:
     1. Pega a série real de valores alvo para uma coluna e a série prevista de valores para essa coluna
     2. Calcula o sMAPE
     3. Retorna o sMAPE para coluna
    """

    error = np.abs(real - prediction)
    scale = (np.abs(real) + np.abs(prediction)) / 2
    smape = (error / scale).mean() * 100

    return smape

- Cálculo da função Final sMAPE:

$$sMAPE final = 25\% * sMAPE(rougher) + 75\% * sMAPE(final)$$

In [ ]:
# Function to final smape evaluation metric


def calculate_final_smape(y_true_rougher, y_pred_rougher, y_true_final, y_pred_final):
    """
    This function:
    1. Pega os valores smapes das variáveis rougher.output.recovery & final.output.recovery
    2. Calcula o final smape
    3. Retorna o final smape
    """
    smape_rougher = calculate_smape(y_true_rougher, y_pred_rougher)
    smape_final = calculate_smape(y_true_final, y_pred_final)
    final_smape = (0.25 * smape_rougher) + (0.75 * smape_final)

    return final_smape

Vamos criar o score da função calculat_smape que foi definida acima

In [ ]:
smape_score = make_scorer(calculate_final_smape, greater_is_better=False)

### 3.2 Treinando diferentes modelos

In [ ]:
def train_fit_score(model, params, name):
    # Criar o objeto GridSearchCV
    grid_search = GridSearchCV(model, params, scoring=smape_score, cv=5, verbose=0)

    # Ajustar o modelo aos dados de treino
    grid_search.fit(features_train, target_train)

    # Fazer previsões nos dados de teste
    predictions = grid_search.predict(features_test)

    # Calcular métricas de avaliação
    rmse = root_mean_squared_error(target_test, predictions)
    mae = mean_absolute_error(target_test, predictions)

    # Calcular o sMAPE para as colunas rougher.output.recovery e final.output.recovery
    smape_rougher = calculate_smape(
        target_test["rougher.output.recovery"], predictions[:, 0]
    )
    smape_final = calculate_smape(
        target_test["final.output.recovery"], predictions[:, 1]
    )

    # Calcular o sMAPE final usando a função calculate_final_smape
    final_smape = calculate_final_smape(
        target_test["rougher.output.recovery"],
        predictions[:, 0],
        target_test["final.output.recovery"],
        predictions[:, 1],
    )

    # Imprimir os melhores parâmetros encontrados pelo GridSearchCV
    print(f"Melhores Parâmetros para {name}:", grid_search.best_params_)

    # Imprimir as métricas de avaliação
    print(f"Root Mean Squared Error (RMSE) para {name}: {rmse:.4f}")
    print(f"Mean Absolute Error (MAE) para {name}: {mae:.4f}")

    print(f"sMAPE para rougher.output.recovery: {smape_rougher:.4f}")
    print(f"sMAPE para final.output.recovery: {smape_final:.4f}")
    print(f"Final sMAPE: {final_smape:.4f}")

    # Retornar as métricas
    return rmse, mae, final_smape

##### 3.2.1 LinearRegression

In [ ]:
%%time
# Definir o modelo
lr_model = LinearRegression()

# Definir a grade de hiperparâmetros para o GridSearchCV
lr_params = {}

# Chamar a função para treinar, ajustar e avaliar o modelo
rmse_LR, mae_LR, final_smape_LR = train_fit_score(
    lr_model, lr_params, "Linear Regression"
)

##### 3.2.2 DecisionTreeRegressor

In [ ]:
%%time
# Definir o modelo
dtr_model = DecisionTreeRegressor(random_state=12345)

# Definir a grade de hiperparâmetros para o GridSearchCV
dtr_params = {"max_depth": [None, 5, 10, 15, 20]}

# Chamar a função para treinar, ajustar e avaliar o modelo
rmse_DTR, mae_DTR, final_smape_DTR = train_fit_score(
    dtr_model, dtr_params, "Decision Tree Regressor"
)

##### 3.2.3 RandomForestRegressor

In [ ]:
%%time
# Definir o modelo
rf_model = RandomForestRegressor(random_state=12345)

# Definir a grade de hiperparâmetros para o GridSearchCV
rf_params = {
    "n_estimators": [10, 20],
    "max_depth": [None, 5, 10, 15],
}

# Chamar a função para treinar, ajustar e avaliar o modelo
rmse_RFR, mae_RFR, final_smape_RFR = train_fit_score(
    rf_model, rf_params, "Random Forest Regressor"
)

## 4. CONCLUSÃO

In [ ]:
# Criar um dicionário com os resultados
results = {
    "Model": [
        "Linear Regression",
        "Decision Tree Regressor",
        "Random Forest Regressor",
    ],
    "RMSE": [rmse_LR, rmse_DTR, rmse_RFR],
    "MAE": [mae_LR, mae_DTR, mae_RFR],
    "Final sMAPE": [final_smape_LR, final_smape_DTR, final_smape_RFR],
}

# Criar um DataFrame a partir do dicionário
results_df = pd.DataFrame(results)

# Imprimir a tabela
results_df


Vamos interpretar os resultados das métricas para cada modelo:

1. Linear Regression:

- RMSE (Root Mean Squared Error): 6.734879
- MAE (Mean Absolute Error): 4.844372
- Final sMAPE: 7.584517

O modelo de Regressão Linear apresenta um desempenho relativamente bom, com valores baixos de RMSE e MAE, indicando que as previsões estão próximas dos valores reais. O valor do Final sMAPE também é baixo, sugerindo que o modelo tem uma boa capacidade de previsão.

2. Decision Tree Regressor:

- RMSE: 10.966146
- MAE: 8.036983
- Final sMAPE: 13.920446

O Decision Tree Regressor apresenta valores mais altos de RMSE, MAE e Final sMAPE em comparação com a Regressão Linear. Isso indica que o modelo de árvore de decisão tem um desempenho inferior na previsão dos dados em comparação com a Regressão Linear.

3. Random Forest Regressor:

- RMSE: 6.811704
- MAE: 5.108131
- Final sMAPE: 8.072978

O Random Forest Regressor apresenta métricas semelhantes à Regressão Linear, indicando um desempenho sólido. O RMSE e o MAE são baixos, sugerindo previsões precisas, e o Final sMAPE também é baixo, indicando uma boa capacidade de generalização.

Conclusões:

- A Regressão Linear e o Random Forest Regressor mostraram desempenhos comparativamente bons.

- O Decision Tree Regressor teve um desempenho inferior em comparação com os outros dois modelos, com valores mais altos de todas as métricas.

- Ao considerar o Final sMAPE, que combina as métricas para ambas as saídas ("rougher.output.recovery" e "final.output.recovery"), o Linear Regression parece ser uma escolha mais sólida.